<a href="https://colab.research.google.com/github/sindhu819/Spark-2020/blob/main/Copy_of_Spark_SQL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession,SQLContext, Row
from operator import add, sub
from time import sleep

from pyspark.streaming import StreamingContext
spark = SparkSession.builder.master("local[*]").getOrCreate()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [75.1 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [6,781 B]
Get:14 http://security.ubuntu.com/ubuntu bionic-security/main am

Congrats! Your Colab is ready to run Pyspark. 

# EDA

We will use a Uber ride dataset for this exercise. Load the dataset into your Colab directory from your local system:

In [ ]:
#1 Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#2. Get the file
#make sure you upload all your data files to your Google drive and change share->Advanced->change->anyone with the link can view
downloaded = drive.CreateFile({'id':'1M8WiN-TSMIBxTK-rSoKdkWdhS-nWKFJe'}) # replace the id with id of file you want to access
downloaded.GetContentFile('Uber-Jan-Feb-FOIL.csv')

In [ ]:
# read the csv file in as a Spark dataframe
df = spark.read.csv('Uber-Jan-Feb-FOIL.csv',inferSchema=True, header =True)

In [ ]:
#Let's preview 10 rows of data
df.show(10)

+-----------------------+--------+---------------+-----+
|dispatching_base_number|    date|active_vehicles|trips|
+-----------------------+--------+---------------+-----+
|                 B02512|1/1/2015|            190| 1132|
|                 B02765|1/1/2015|            225| 1765|
|                 B02764|1/1/2015|           3427|29421|
|                 B02682|1/1/2015|            945| 7679|
|                 B02617|1/1/2015|           1228| 9537|
|                 B02598|1/1/2015|            870| 6903|
|                 B02598|1/2/2015|            785| 4768|
|                 B02617|1/2/2015|           1137| 7065|
|                 B02512|1/2/2015|            175|  875|
|                 B02682|1/2/2015|            890| 5506|
+-----------------------+--------+---------------+-----+
only showing top 10 rows



In [ ]:
#How many rows are in this dataset
df.count()

354

In [ ]:
len(df.columns)

4

In [ ]:
# What's the column description?
df.printSchema()

root
 |-- dispatching_base_number: string (nullable = true)
 |-- date: string (nullable = true)
 |-- active_vehicles: integer (nullable = true)
 |-- trips: integer (nullable = true)



In [ ]:
df.printSchema()

root
 |-- dispatching_base_number: string (nullable = true)
 |-- date: string (nullable = true)
 |-- active_vehicles: integer (nullable = true)
 |-- trips: integer (nullable = true)



In [ ]:
# Find out if there is any null values. If so, how many null values are present in each column?
 from pyspark.sql.functions import isnan, when, count, col 
 df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+-----------------------+----+---------------+-----+
|dispatching_base_number|date|active_vehicles|trips|
+-----------------------+----+---------------+-----+
|                      0|   0|              0|    0|
+-----------------------+----+---------------+-----+



In [ ]:
#How many, for each distinct data in the dispatching_base_number column, are there?
df.groupby('dispatching_base_number').count().show()

+-----------------------+-----+
|dispatching_base_number|count|
+-----------------------+-----+
|                 B02512|   59|
|                 B02598|   59|
|                 B02682|   59|
|                 B02765|   59|
|                 B02617|   59|
|                 B02764|   59|
+-----------------------+-----+



In [ ]:
df.createOrReplaceTempView("uber")

In [ ]:
spark.sql("select distinct(dispatching_base_number) as base, count(*) from uber group by (dispatching_base_number)").show()

+------+--------+
|  base|count(1)|
+------+--------+
|B02682|      59|
|B02598|      59|
|B02512|      59|
|B02764|      59|
|B02765|      59|
|B02617|      59|
+------+--------+



In [ ]:
# Register as table ready for Spark SQL 
df.registerTempTable("uber1")

In [ ]:
# Show with Spark SQL, what are the distinct values of dispatching_base_number?
k=spark.sql("select distinct(dispatching_base_number) from uber")

In [ ]:
for i in k.collect():
  print(i)

Row(dispatching_base_number='B02512')
Row(dispatching_base_number='B02598')
Row(dispatching_base_number='B02682')
Row(dispatching_base_number='B02765')
Row(dispatching_base_number='B02617')
Row(dispatching_base_number='B02764')


In [ ]:
spark.sql("select distinct(dispatching_base_number) from uber").show()

+-----------------------+
|dispatching_base_number|
+-----------------------+
|                 B02512|
|                 B02598|
|                 B02682|
|                 B02765|
|                 B02617|
|                 B02764|
+-----------------------+



#### Show, for each distinct value of dispatching_base_number, the total number of trips made. Display the result from high to low with respect to the total number of trips made.

In [ ]:
result = spark.sql("""select distinct(dispatching_base_number) as base,sum(trips) as cnt from uber group by dispatching_base_number order by cnt desc""")

In [ ]:
result.show()

+------+-------+
|  base|    cnt|
+------+-------+
|B02764|1914449|
|B02617| 725025|
|B02682| 662509|
|B02598| 540791|
|B02765| 193670|
|B02512|  93786|
+------+-------+



In [ ]:
type(result)

pyspark.sql.dataframe.DataFrame

In [ ]:
# Filter the above result and show only those with the total number of trips made greater than 700,000
result.filter(result.cnt>700000).show()

+------+-------+
|  base|    cnt|
+------+-------+
|B02764|1914449|
|B02617| 725025|
+------+-------+



In [ ]:
result.registerTempTable("result")
spark.sql("""select * from result where cnt >700000""").show()

+------+-------+
|  base|    cnt|
+------+-------+
|B02764|1914449|
|B02617| 725025|
+------+-------+



In [ ]:
# Repeat by using the Spark SQL
result.registerTempTable("result")
spark.sql("""select * from result where cnt >700000""").show()

+------+-------+
|  base|    cnt|
+------+-------+
|B02764|1914449|
|B02617| 725025|
+------+-------+



####Show the total number of trips made for each distinct date.   
####Dispay only the top 5 highest results of the total number of trips made and show the related date.

In [ ]:
spark.sql("""select date ,sum(trips) as cnt from uber group by date order by cnt desc""").show(5)

+---------+------+
|     date|   cnt|
+---------+------+
|2/20/2015|100915|
|2/14/2015|100345|
|2/21/2015| 98380|
|2/13/2015| 98024|
|1/31/2015| 92257|
+---------+------+
only showing top 5 rows

